In [ ]:
# https://github.com/Nixtla/neuralforecast/blob/main/nbs/models.deepar.ipynb
# https://github.com/Nixtla/neuralforecast/blob/main/nbs/losses.pytorch.ipynb

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.distributions import NegativeBinomial, Normal, Poisson

from bikes.preprocess.preprocess import get_tensor_train_dataset, Scaler
from bikes.evaluate.split import train_test_split

In [3]:
def plot_timeseries(actual: pd.Series, predicted: pd.Series):
    fig, ax = plt.subplots()

    ax.plot(actual, label="Observed")
    ax.plot(predicted, label="Predicted")

    ax.set(ylabel="Count")
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)
    ax.legend()

    fig.tight_layout();

    return ax

## DeepAR

### Distribution Loss

In [ ]:
class NormalDistributionLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def scale_params(
        self,
        mean: torch.Tensor,
        std: torch.Tensor,
        loc: torch.Tensor | None = None,
        scale: torch.Tensor | None = None,
    ) -> tuple[torch.Tensor, torch.Tensor]:
        std = F.softplus(std)
        if (loc is not None) and (scale is not None):
            mean = (mean * scale) + loc
            std = (std + 0.2) * scale
        return mean, std

    def __call__(
        self,
        output: torch.Tensor,
        y: torch.Tensor,
        loc: torch.Tensor | None = None,
        scale: torch.Tensor | None = None,
    ):
        mean, std = torch.tensor_split(output, 2, dim=2)
        mean, std = self.scale_params(mean, std, loc, scale)
        loss_dist = Normal(loc=mean, scale=std)
        return (-loss_dist.log_prob(y)).sum()

In [8]:
type(torch.rand(1))

torch.Tensor

In [ ]:
cycle_counts = pd.read_csv("cycle_counts.csv", parse_dates=["date"])

In [ ]:
LOCATION = "Quay Street Eco Display Classic"

location_df = cycle_counts.loc[cycle_counts["location"] == LOCATION].copy()
location_df = location_df.set_index("date").sort_index()
train_df, test_df = train_test_split(location_df)
y_train, y_test = train_df["count"], test_df["count"]

In [ ]:
fig, ax = plt.subplots()
ax.plot(y_train.iloc[-500:], label="Observed", lw=2)
ax.set(ylabel="Count")
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
ax.legend()
fig.tight_layout();